[sklearn API: dcg](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.dcg_score.html#sklearn.metrics.dcg_score)  
[dcg](https://github.com/scikit-learn/scikit-learn/blob/0d378913be6d7e485b792ea36e9268be31ed52d0/sklearn/metrics/_ranking.py#L1362)  
[_tie_averaged_dcg](https://github.com/scikit-learn/scikit-learn/blob/0d378913be6d7e485b792ea36e9268be31ed52d0/sklearn/metrics/_ranking.py#L1299)

In [ ]:
import numpy as np
import pandas as pd

from sklearn.metrics import ndcg_score, dcg_score

In [6]:
num_samples = 5
num_items = 7

rng = np.random.default_rng(42)
y_true = rng.integers(1,5, (num_samples, num_items))

rng = np.random.default_rng(44)
y_score = rng.integers(1,5, (num_samples, num_items))

$ \large \mathrm{DCG_{p}} = \sum_{i=1}^{p} \frac{rel_{i}}{\log_{2}{(i+1)}}$

$ \large \mathrm{nDCG_{p}} = \frac{\mathrm{DCG_{p}}}{\mathrm{IDCG_{p}}}$, where $\mathrm{IDCG_{p}} = \sum_{i=1}^{|REL_{p}|} \frac{rel_{i}}{\log_{2}{(i+1)}}$

In [7]:
def dcg_array_ignore_ties(y_true, y_score, k=None, log_base=2):
    discount = 1 / (np.log(np.arange(y_true.shape[1]) + 2) / np.log(log_base))
    if k is not None:
        discount[k:] = 0
        
    ranking = np.argsort(y_score)[:, ::-1]
    ranked = y_true[np.arange(ranking.shape[0])[:, np.newaxis], ranking]
    return discount.dot(ranked.T)

validation for dcg score (ignoring ties)

In [9]:
dcg_array_ignore_ties(y_true, y_score)

array([ 8.94781187, 10.77283194,  9.90113586, 10.92878593,  8.45482712])

In [8]:
dcg_array_ignore_ties(y_true, y_score).mean() == dcg_score(y_true, y_score, ignore_ties=True)

True

validation for ndcg score (ignoring ties)

In [10]:
ncdg_array = dcg_array_ignore_ties(y_true, y_score) / dcg_array_ignore_ties(y_true, y_true)
ncdg_array.mean() == ndcg_score(y_true, y_score, ignore_ties=True)

True

In [ ]:
discount_cumsum = np.cumsum(discount)

_, inv, counts = np.unique(-y_score[0], return_inverse=True, return_counts=True)
ranked = np.zeros(len(counts))
np.add.at(ranked, inv, y_true[0])
ranked /= counts
groups = np.cumsum(counts) - 1
discount_sums = np.empty(len(counts))
discount_sums[0] = discount_cumsum[groups[0]]
discount_sums[1:] = np.diff(discount_cumsum[groups])

(ranked * discount_sums).sum()